### 1. Setting up the Environment

In [1]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import google.generativeai as genai
import base64
import pickle
import requests

d:\My Projects\MailSqueeze\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Load the env

In [2]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


### 3. Gmail Authentication

In [3]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_gmail_service():
    creds = None
    if os.path.exists('tokens.json'):
        creds = Credentials.from_authorized_user_file('tokens.json', SCOPES)
        
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
            
        with open('tokens.json', 'w') as token:
            token.write(creds.to_json())
            
    service = build('gmail', 'v1', credentials=creds)
    print("Gmail service created successfully.")
    return service

### 4. Fetch unread email IDs

In [4]:
def get_unread_emails(service, max_results=5):
    results = service.users().messages().list(
        userId='me', 
        labelIds=['INBOX'], 
        q='is:unread', 
        maxResults=max_results
        ).execute()
    return results.get('messages', [])


### 5. Extract email content

In [5]:
def get_email_body(service, msg_id):
    msg = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
    parts = msg['payload'].get('parts', [])
    for part in parts:
        if part['mimeType'] == 'text/plain':
            data = part['body']['data']
            text = base64.urlsafe_b64decode(data).decode('utf-8')
            return text
    return "No readable content found."


### 6. Summarize with Gemini

In [11]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

print("Available models:")
for model in genai.list_models():
    print(f"- {model.name} : {model.description}")


Available models:
- models/embedding-gecko-001 : Obtain a distributed representation of a text.
- models/gemini-1.0-pro-vision-latest : The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
- models/gemini-pro-vision : The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
- models/gemini-1.5-pro-latest : Alias that points to the most recent production (non-experimental) release of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens.
- models/gemini-1.5-pro-002 : Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 million tokens, released in September of 2024.
- models/gemini-1.5-pro : Stable version of Gemini 1.5 Pro, our mid-size multimodal model that supports up to 2 mil

In [12]:

model = genai.GenerativeModel('models/gemini-2.5-flash')

def summarize_with_gemini(text):
    prompt = f"Summarize this email:\n\n{text}"
    response = model.generate_content(prompt)
    return response.text.strip()


### 7. Send to Telegram

In [8]:

def send_to_telegram(message):
    token = os.getenv('TELEGRAM_BOT_TOKEN')
    chat_id = os.getenv('TELEGRAM_CHAT_ID')
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    data = {'chat_id': chat_id, 'text': message}
    
    try:
        response = requests.post(url, data=data)
        response.raise_for_status()  # Raise error for bad HTTP status
        
        result = response.json()
        if not result.get('ok'):
            print("Telegram API returned an error:", result)
            # You can also raise an exception or log it here
            
        else:
            print("Message sent successfully!")

    except requests.exceptions.RequestException as e:
        print("Request failed:", e)
        # Handle network errors, timeouts, etc.


### 8. Test the Flow

In [13]:
service = get_gmail_service()
messages = get_unread_emails(service)

if not messages:
    print("No unread emails found.")
else:
    for msg in messages:
        text = get_email_body(service, msg['id'])
        summary = summarize_with_gemini(text)
        print("Summary:\n", summary)
        send_to_telegram(f"📧 New Email Summary:\n{summary}")


Gmail service created successfully.
Summary:
 This is a security alert from Google notifying nimsaraekanayakayu@gmail.com that a third-party application called MailSqueeze has been granted access to their Google Account data.

The email advises the user to take action if they did not authorize this access, as it could indicate unauthorized activity. It provides links to check account activity, revoke MailSqueeze's access, and review overall security.
Request failed: 400 Client Error: Bad Request for url: https://api.telegram.org/bot7878635317:AAHymlljGL59t5DaCX_RAHoSAat-FCx9Xfo/sendMessage
Summary:
 This email announces "Win A Lambo Round 12," a contest where participants can win a Lamborghini by obtaining a Golden Ticket.

There are four ways to acquire Golden Tickets:
1.  **Play the game:** Earn 1 ticket for every 0.005 BTC wagered on Multiply BTC/Betting.
2.  **Buy tickets:** Purchase them for 0.00025 BTC each.
3.  **Wheel of Fortune:** Use free spin credits (from emails) to win up 